In [3]:
%useLatestDescriptors
%use coroutines
USE {
  repositories {
    mavenLocal()
    mavenCentral()
  }
  dependencies(
    "me.kpavlov.aimocks:ai-mocks-openai-standalone:0.5.0-Beta02",
    "io.kotest:kotest-assertions-core:5.9.1",
    "dev.langchain4j:langchain4j-open-ai:1.5.0",
    "dev.langchain4j:langchain4j-kotlin:1.5.0-beta11",
  )
}

In [4]:
import me.kpavlov.aimocks.openai.MockOpenai
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.model.chat.chat
import dev.langchain4j.model.openai.OpenAiChatModel
import dev.langchain4j.model.openai.OpenAiChatRequestParameters
import dev.langchain4j.model.output.FinishReason
import io.kotest.matchers.shouldBe
import io.kotest.matchers.shouldNotBe

val openai = MockOpenai(verbose = true, port = 0)

val model: OpenAiChatModel =
  OpenAiChatModel
    .builder()
    .apiKey("foo")
    .baseUrl(openai.baseUrl())
    .build()

openai.completion {
//  temperature(0.42) = 0.42
//  seed = 100500
  model = "4o"
  maxTokens = 120
} responds {
  assistantContent = "Hello"
  finishReason = "stop"
}

runBlocking {

  val result =
    model.chat {
      parameters =
        OpenAiChatRequestParameters
          .builder()
          .maxCompletionTokens(120)
          .temperature(0.42)
          .modelName("4o")
          .seed(100500)
          .build()
      messages += userMessage("Say Hello")
    }

  result.apply {
    finishReason() shouldBe FinishReason.STOP
    tokenUsage() shouldNotBe null
    aiMessage().text() shouldBe "Hello"
  }

  println("OpenAI Response: ${result.aiMessage().text()}")
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[4], line 3, column 35: Unresolved reference: chat
at Cell In[4], line 10, column 14: None of the following functions can be called with the arguments supplied: 
public constructor MockOpenai() defined in me.kpavlov.aimocks.openai.MockOpenai
public constructor MockOpenai(port: Int, verbose: Boolean) defined in me.kpavlov.aimocks.openai.MockOpenai
at Cell In[4], line 20, column 3: Unresolved reference: temperature
at Cell In[4], line 21, column 3: Unresolved reference: seed
at Cell In[4], line 22, column 3: Val cannot be reassigned
at Cell In[4], line 22, column 11: Type mismatch: inferred type is String but OpenAiChatModel was expected
at Cell In[4], line 23, column 3: Unresolved reference: maxTokens
at Cell In[4], line 25, column 3: Unresolved reference: assistantContent
at Cell In[4], line 26, column 3: Unresolved reference: finishReason
at Cell In[4], line 32, column 11: None of the following functions can be 